<a href="https://colab.research.google.com/github/iisuleymanogluvural/iisuleymanogluvural/blob/main/stuttersep28k_Mfcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
bschuss02_sep28k_path = kagglehub.dataset_download('bschuss02/sep28k')
mitulgargsam_sep28kmfcc_path = kagglehub.dataset_download('mitulgargsam/sep28kmfcc')

print('Data source import complete.')


Asıl hedefim, Sounddevice ile anlık kekemelik türünün tespitini sağlamak
o yüzden sounddevice kütüphanesini ,kagge'da kullanamasam da install yaptım

In [ ]:
pip install sounddevice librosa scikit-learn numpy

In [ ]:
#I performed version control because I will be using lightGBM for machine learning.
import lightgbm
print(lightgbm.__version__)

In [ ]:

import pandas as pd #dataframe ,Data analysis
import numpy as np #Numerical Computing,MultiDimensional Array Core library
from sklearn.model_selection import train_test_split #ML,Selects train/Test data
from sklearn.preprocessing import LabelEncoder#convert categorical labels into numerical form.
from lightgbm import LGBMClassifier#effective gradient boosting-based classification model
import joblib#save and load models to and from disk


In [ ]:
df = pd.read_csv("/kaggle/input/sep28kmfcc/sep28k-mfcc.csv")  # dosyayı okuduk
df = df.dropna()#NaN Data had been removed

print(df.columns.tolist())#Control data head


In [ ]:
#I needed to see the beginning of the data, it appears with df.head.
print(df.head())

In [ ]:
#Since the data is multitable, we took the part where each column equals 1.
def assign_label(row):
    if row['Prolongation'] == 1:
        return 'Prolongation'
    elif row['Block'] == 1:
        return 'Block'
    elif row['SoundRep'] == 1:
        return 'SoundRep'
    else:
        return 'Other'

df['label'] = df.apply(assign_label, axis=1)


In [ ]:
# Target label : 'label' (example: Prolongation, Block, SoundRep)
df = df[df['label'].isin(['Prolongation', 'Block', 'SoundRep'])]


In [ ]:
#fitting,With LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])


In [ ]:
# Exclude cols that will not be used for model
exclude_cols = ['Show', 'Name', 'label', 'label_encoded']
feature_cols = [col for col in df.columns if col not in exclude_cols]

X = df[feature_cols]
y = df['label_encoded']


In [ ]:
#Model Train with LGBMClassifier
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

model = LGBMClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))


In [ ]:
#Save Model
import joblib

joblib.dump(model, 'stutter_model.pkl')


In [ ]:

joblib.dump(le, 'label_encoder.pkl')


In [ ]:
joblib.dump(feature_cols, 'feature_cols.pkl')


In [ ]:
#Control again
print (df.columns.tolist())

In [ ]:
# By removing those that will not be used from the model from all columns:
all_columns = df.columns.tolist()
exclude_cols = ['Show', 'Name', 'label', 'label_encoded'] #We dont use
feature_cols = [col for col in all_columns if col not in exclude_cols]

print("Feature numbers,will be used in model:", len(feature_cols)) #control again
print(feature_cols)


In [ ]:
#for voice normlization to 16 Khz
import librosa
import soundfile as sf

def normalize_audio_to_16k(file_path, save_path=None):
    # 16 kHz olarak yükle
    y, sr = librosa.load(file_path, sr=16000)

    if save_path:
        sf.write(save_path, y, 16000)

    return y, 16000

# Kullanım:
y, sr = normalize_audio_to_16k("/kaggle/input/sep28k/clips/stuttering-clips/clips/FluencyBank_010_0.wav")
print(f"Audio loaded with sampling rate: {sr}")

In [ ]:
import librosa
import numpy as np #Looked to Chatgpt
def extract_features(file_path, trim_silence=True, sr=22050):
    # Ses dosyasını yükle
    y, sr = librosa.load(file_path, sr=sr)


    # Normalize the voice data
    if np.max(np.abs(y)) > 0:
        y = y / np.max(np.abs(y))

    # MFCC
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs, axis=1)

    # Chroma
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = np.mean(chroma, axis=1)

    # Zero Crossing Rate
    zcr = np.mean(librosa.feature.zero_crossing_rate(y))

    # Spectral Contrast
    spec_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr))

    # Hepsini birleştir
    features = np.hstack([mfccs_mean, chroma_mean, zcr, spec_contrast])
    return features


In [ ]:
features = extract_features("/kaggle/input/sep28k/clips/stuttering-clips/clips/FluencyBank_010_0.wav")
print(features.shape)  # 29-2 (exported labelencoder,label x entrance)

In [ ]:
import joblib
import pandas as pd

# upload Model,Encoder,Column Names
model = joblib.load("stutter_model.pkl")
le = joblib.load("label_encoder.pkl")
feature_cols = joblib.load("feature_cols.pkl")

def predict_from_audio(file_path):
    features = extract_features(file_path)

    # fill with zeros if column's number doesnt match
    if len(features) < len(feature_cols):
        features = np.append(features, np.zeros(len(feature_cols) - len(features)))
    elif len(features) > len(feature_cols):
        features = features[:len(feature_cols)]

    # transform the data appropriately
    features_df = pd.DataFrame([features], columns=feature_cols)

    pred = model.predict(features_df)
    label = le.inverse_transform(pred)

    print("Predicted Stuttering Type:", label[0])
    return label[0]


In [ ]:
print(df.columns.tolist())

In [ ]:
"""I am having a problem here. I mean, it calls all stuttering "block"! The reason is, I don't understand its reason!!!"""

In [ ]:
predict_from_audio("/kaggle/input/sep28k/clips/stuttering-clips/clips/FluencyBank_010_89.wav")